In [ ]:
import os, sys, import_ipynb, ipynb
if os.getcwd() == 'C:\\Users\\admin\\Desktop\\retail_data_analysis\\code':
    print ("Already in code directory")
else:
    os.chdir('..//code')
        
# import python libraries
from importLibraries import *


def fun_lda():   
    
    os.chdir('..//input')
    input_data_preProcessed = pd.read_csv("input_data_preProcessed.csv")
    input_data_preProcessed_Description = input_data_preProcessed[['Description']]
    
    
    input_data_preProcessed_Description['Description'] = input_data_preProcessed_Description['Description'].apply(lambda x: "".join(x) )
    input_data_preProcessed_Description = input_data_preProcessed_Description['Description'].tolist()
    
    # LDA can only use raw term counts (not tfidf) because its a PGM
    tf_vectorizer = CountVectorizer(max_df=0.9, min_df=25, analyzer='word', ngram_range=(1, 2), max_features=1000, stop_words='english')
    tf = tf_vectorizer.fit_transform(input_data_preProcessed_Description)
    tf_feature_names = tf_vectorizer.get_feature_names()
    

    # Run LDA
    lda = LatentDirichletAllocation(n_components = 4, max_iter=2, learning_method='online', learning_offset=50.).fit(tf)
    
    
    def display_topics(model, feature_names, no_top_words):
        topic_dict = {}
        for topic_idx, topic in enumerate(model.components_):
            topic_dict["Topic %d words" % (topic_idx)]= ['{}'.format(feature_names[i]) for i in topic.argsort()[:-no_top_words - 1:-1]]
            topic_dict["Topic %d weights" % (topic_idx)]= ['{:.1f}'.format(topic[i]) for i in topic.argsort()[:-no_top_words - 1:-1]]
        return pd.DataFrame(topic_dict)

    no_top_words = 5
    lda_result = display_topics(lda, tf_feature_names, no_top_words)
    display(lda_result)
    display("Perplexity of this model is: ", lda.perplexity(tf))
    
    # Optimal number of topics using grid search
    # One way to find the optimal number of topics is to train with different values of alpha and beta and test perplexity on test data and whichever alpha, beta gives minimum perplexity on test data choose that value of alpha and beta
# To determine the best model, GridSearchCV uses the approximate log-likelihood as score (not perplexity). 
    # One way to find the optimal number of topics is to train with different values of alpha and beta and test perplexity on test data and whichever alpha, beta gives minimum perplexity on test data choose that value of alpha and beta
    search_params = {'n_components': [4, 5, 6], 'learning_decay': [.5, .7]}
    # Init the Model
    lda = LatentDirichletAllocation(max_iter=2, learning_method='online', batch_size=128, evaluate_every = -1,  n_jobs = -1)
    # Init Grid Search Class
    model = GridSearchCV(lda, param_grid=search_params)
    # Do the Grid Search
    model.fit(tf)
    # Best Model
    best_lda_model = model.best_estimator_
    # Model Parameters
    print("Best Model's Params: ", model.best_params_)
    # Log Likelihood Score
    print("Best Log Likelihood Score: ", model.best_score_)
    # Perplexity
    print("Model Perplexity: ", best_lda_model.perplexity(tf))


